In [1]:
import numpy as np
import scipy.io
from scipy.stats import kurtosis
from scipy.stats import skew
from scipy.signal import savgol_filter
from scipy.signal import find_peaks
from scipy.signal import butter, lfilter
from scipy.signal import butter, sosfilt, sosfreqz
from scipy.signal import welch
from scipy import signal
import os
import pandas as pd

In [2]:
import matplotlib.pyplot as plt

In [3]:
save_path = 'D:\EECE499\Features\GSRFeatures.xlsx'
#save_path = '..\..\GSRFeatures.xlsx'

In [4]:
sample_rate = 128

In [5]:
def butter_bandpass(lowcut, highcut, fs, order):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    sos = butter(order, [low, high], analog=False, btype='band', output='sos')
    return sos

def butter_bandpass_filter(data, lowcut, highcut, fs, order=6):
    sos = butter_bandpass(lowcut, highcut, fs, order=order)
    y = sosfilt(sos, data)
    return y

def butter_lowpass(cutoff, fs, order):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=6):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y

In [6]:
max_length = 17215

In [7]:
data_frames = []

for user_id in range(1, 59):
    print('User ID: ' + str(user_id))
    
    for clip_id in range(1, 37):
        #print('Clip ID: ' + str(clip_id))
        
        #D:\EECE499\Raw\MyECGFunc\ASCERTAIN_Raw
        data_path = 'D:/EECE499/Raw/MyECGFunc/ASCERTAIN_Raw/GSRData/Movie_P' + str(user_id).zfill(2) + '/GSR_Clip' + str(clip_id) + '.mat'    
        #data_path = './../ASCERTAIN_Raw/GSRData/Movie_P' + str(user_id).zfill(2) + '/GSR_Clip' + str(clip_id) + '.mat'
        
        presentation_id.append(str(user_id).zfill(2) + str(clip_id).zfill(2))
        
        if os.path.isfile(data_path):
            
            signal = scipy.io.loadmat(data_path)
            resistance = signal['Data_GSR'][:,4]
            time_stamp = signal['Data_GSR'][:,0]
            
            resistance = np.array(resistance)

User ID: 1


NameError: name 'presentation_id' is not defined